# Install

In [ ]:
import os

# check if `pytorch-CycleGAN-and-pix2pix` is already cloned
if not os.path.exists('pytorch-CycleGAN-and-pix2pix'):
	!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
%pip install -q -r requirements.txt

# Testing

test the model with single mode

In [ ]:
!ls checkpoints/

In [ ]:
! python test.py --dataroot ./datasets/test_pose/testA --name fish_pix2pix --model test --netG unet_256 --direction AtoB --dataset_mode single --norm batch --num_test 10000

# Combine all the generated images into a video

In [ ]:
import cv2
from PIL import Image
import numpy as np

images_path = 'results/fish_pix2pix/test_latest/images/'
output_path = 'results/fish_pix2pix/videos/'
output_width, output_height = 1280, 720

# check if the output path exists, if not create it
if not os.path.exists(output_path):
    os.makedirs(output_path)

image_array = os.listdir(images_path)
# only keep the images that with 'fake_B' in the name
image_array = [x for x in image_array if '_fake' in x]
image_array.sort(key=lambda x: int(x.split('_')[0]))
# print(image_array)

img_array = []
for filename in image_array:
    img = Image.open(f'{images_path}/{filename}')
    # resize the image
    img = img.resize((output_width, output_height), resample=Image.BICUBIC)
    img = np.array(img)
    img_array.append(img)

print(f"Number of frames: {len(img_array)}")
print(f"Frame size: {(output_width, output_height)}")
# print(img_array)

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip

clip = ImageSequenceClip(img_array, fps=30)  # Adjust fps as needed

# set audio
audio = AudioFileClip('../src/magnetic.mp4')

clip = clip.set_audio(audio)

clip.write_videofile(f'../src/magnetic_fish.mp4', codec='libx264', audio_codec="aac", fps=30)

# Combine the video

combine 3 imgs into a video
- real_world_dance
- real_world_pose
- generated_video

In [ ]:
test_img_folder = 'datasets/test_pose/dance2pose/'
gen_img_array = img_array

# sort the test images
test_img_array = os.listdir(test_img_folder)
test_img_array.sort(key=lambda x: int(x.split('.')[0]))

print(f"Number of test images: {len(test_img_array)}")
print(f"Number of generated images: {len(gen_img_array)}")

# combine the test and generated images
combined_img_array = []
for test_img, gen_img in zip(test_img_array, gen_img_array):
    test_img = Image.open(f'{test_img_folder}/{test_img}')
    gen_img = Image.fromarray(gen_img)
    combined_img = Image.new('RGB', (3*output_width, output_height))
    combined_img.paste(test_img, (0, 0))
    combined_img.paste(gen_img, (2*output_width, 0))
    combined_img = np.array(combined_img)
    combined_img_array.append(combined_img)
    
print(f"Number of combined images: {len(combined_img_array)}")

# Write the video out

In [ ]:
from moviepy.editor import ImageSequenceClip

clip = ImageSequenceClip(combined_img_array, fps=30)  # Adjust fps as needed

audio_clip = AudioFileClip("../src/magnetic.mp4")

clip = clip.set_audio(audio_clip)

clip.write_videofile(f'../src/magnetic_dance2pose2fish.mp4', codec='libx264', audio_codec="aac",  fps=30)